In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import yfinance as yf

from yahoo_fin import stock_info as si

from pandas_datareader import data

In [2]:
def cumm_prod(price_df, start=None, end=None):
    """
    Calculating the cummulative product of the percentage change to normalize the return and be
    able to compare the stocks. 

    """
    if start == None and end == None:
        return_series = (price_df.pct_change()+1).cumprod() - 1
        return_series.dropna(inplace=True)
    else:
        if end == None:
            end = date.today()
        return_series = (price_df.loc[start:end].pct_change()+1).cumprod() - 1
        return_series.dropna(inplace=True)
        
    
    return return_series

def annualized_return(adjClose_return_df, n_years=None):
    """
    n_years needs to represent the period of the adjClose_return_df.
        If it's 1 year worth of cummulative return, you input 1 for the 1y annualized returns.
        If it's 3 years worth of cummulative return, you input 3 for the long period annualized returns.

        (1+total return)pow(1/N) - 1) where N = number of years
    """
    if n_years == None or n_years == 0:
        print("Input How many years does your data covers!")
    else:
        annualized_return = ((1 + adjClose_return_df.tail(1))**(1/n_years)-1) * 100
        return annualized_return

In [3]:
def market_cap_cat(market_cap):
    try:
        if market_cap > 10000000000:
            market_cap_cat = 'Large'
        elif 2000000000 < market_cap < 1000000000:
            market_cap_cat = 'Medium'
        else:
            market_cap_cat = 'Small'
        return market_cap_cat
    except:
        return 'NaN'

def earnings_expectations(FPE, TPE):
    try:
        if FPE > TPE:
            expect = 'Decrease'
        else:
            expect = 'Increase'
            
        return expect
    except:
        return 'NaN'

def CAPM_analysis(beta, annual_return, market_return, risk_free_rate = 0.0192):
    try:
        capm_status = 'TBD'
        
        capm_ER = risk_free_rate + beta * (market_return - risk_free_rate)

        if annual_return > capm_ER:
            capm_status = 'OverValued'
        else:
            capm_status = 'UnderValued'
        
        return capm_status, capm_ER
    except:
        return 'NaN'
        
def intrinsic_valuation(ticker, eps_ttm, growth_estimate_5y, PE_trailing, target_return, safety_margin=0.5):

    try:
        irv_status = 'TBD'

        forecast_eps_at_year_10 = eps_ttm * ( 1 + growth_estimate_5y) ** 9
        forecast_value_at_year_10 = forecast_eps_at_year_10 * PE_trailing

        pv_future_value = forecast_value_at_year_10/(1 + target_return)**9

        pv_forecast_value_safe = pv_future_value * safety_margin

        if float(si.get_live_price(ticker)) > pv_forecast_value_safe:
            irv_status = 'Above Fair Price'
        else:
            irv_status = 'Below Fair Price'

        return irv_status
    except:
        return 'NaN'

def get_ESG(ticker):
    ticket = yf.Ticker(ticker)
    esg_data = pd.DataFrame(ticket.sustainability)['Value'][['socialScore', 'governanceScore', 'environmentScore', 'totalEsg', 'percentile']]
    
    return esg_data.to_dict()

In [4]:
SPY_adj_close = si.get_data('SPY', start_date= str(date.today().year - 10) +'-'+ str(date.today().month) +'-'+ str(date.today().day), end_date=str(date.today()))['adjclose']
cummulative_prod_SPY = cumm_prod(SPY_adj_close)
annu_return_10y_SPY = annualized_return(cummulative_prod_SPY, 10)[0]

In [5]:
annu_return_10y_SPY

14.365430954774627

In [6]:
def get_data(ticker):

    info = {}

    info['ticker'] = ticker
    info['name'] = si.get_quote_data(ticker)['shortName']
    info['sector'] = si.get_company_info(ticker)['Value']['sector']
    info['industry'] = si.get_company_info(ticker)['Value']['industry']

    adj_close = si.get_data(ticker, start_date= str(date.today().year - 10) +'-'+ str(date.today().month) +'-'+ str(date.today().day), end_date=str(date.today()))['adjclose']
    cummulative_prod = cumm_prod(adj_close)
    annu_return_10y = annualized_return(cummulative_prod, 10)[0]

    info['current_price'] = float(si.get_live_price(ticker))
    info['annu_return_10y'] = annu_return_10y

    info['market_cap'] = si.get_quote_table(ticker)['Market Cap']

    cap = float(si.get_quote_data('AMD')['marketCap'])
    info['marketcap_cat'] = market_cap_cat(cap)


    info['beta'] = float(si.get_quote_table(ticker)['Beta (5Y Monthly)'])

    esg = get_ESG(ticker)
    info['socialScore'] = esg['socialScore']
    info['governanceScore'] = esg['governanceScore']
    info['environmentScore'] = esg['environmentScore']
    info['totalEsg'] = esg['totalEsg']
    info['percentile'] = esg['percentile']
    

    info['eps_ttm'] = float(si.get_quote_table(ticker)['EPS (TTM)'])
    info['growth_estimate_5y'] = float(si.get_analysts_info(ticker)['Growth Estimates'][ticker][4][:-1]) / 100

    # if FPE > TPE => Investor expect a decrease in earnings.
    info['pe_forward'] = float(si.get_quote_data(ticker)['forwardPE'])
    info['pe_trailing'] = float(si.get_quote_data(ticker)['trailingPE'])

    info['Earnings_expect'] = earnings_expectations(info['pe_forward'], info['pe_trailing'])
    info['irv_status'] = intrinsic_valuation(ticker, info['eps_ttm'],  info['growth_estimate_5y'], info['pe_trailing'], 0.15)

    info['capm_status'] = CAPM_analysis(info['beta'], annu_return_10y, annu_return_10y_SPY)[0]



    return info

In [7]:
panel_data = pd.DataFrame(columns=['ticker', 'name', 'sector', 'industry', 'current_price', 'annu_return_10y', 'market_cap', 'marketcap_cat', 'beta', 'eps_ttm', 'growth_estimate_5y', 'pe_forward', 'pe_trailing', 'Earnings_expect', 'irv_status', 'capm_status', 'socialScore', 'governanceScore', 'environmentScore', 'totalEsg', 'percentile'])

In [8]:
sp500_list_50 = si.tickers_sp500()[:50]

In [9]:

for ticker in sp500_list_50:
        try:
                print(ticker)
                info = get_data(ticker)
                panel_data = panel_data.append(info,ignore_index=True)
        except:
                pass
       



A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
AMAT


In [ ]:
panel_data